In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

In [2]:
# parameters
customer_id = '90'
formatted_attribute = 'Tab Count'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=762
attribute_id_input=2425

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [10]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
custom_field_df=pd.json_normalize(df['custom_fields'])
# custom_field_df = pd.json_normalize(df['custom_fields'])
fields = ['Tab Style']
tab = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

2269


In [19]:
dia=r'''(?i)(\d+.?tab)|()'''
tab['eigth']=tab['long_desc'].apply(lambda x: re_extract(dia, str(x)))

tab['eigth']=tab['eigth'].apply(lambda x: re.sub(r"(?i)[\-\s•]?tab|7530009594441|\[\'|\'\]|",'',str(x)))
dia=r'''(?i)(\,|\[)|()'''
tab['nine']=tab['eigth'].apply(lambda x: re_extract(dia, str(x)))
tabs=tab[tab['nine'].astype(str)=='[]']
tabs[curation_col]=tabs['eigth']
matchtab=tabs[['external_id',curation_col]]
matchtab
# tab['nine'].explode().value_counts()

external_id Q:tab_count
19    5637176800           5
20    5637176802          10
21    5637176803          12
22    5637176804          15
23    5637176805          12
24    5637176806          26
25    5637176807          31
43    5637176736          25
44    5637176737          31
45    5637176738          12
46    5637176739           5
47    5637176740           5
48    5637176741           8
49    5637176742           8
50    5637176743          25
51    5637176744          31
52    5637176745          12
53    5637176746           5
54    5637176747           5
55    5637176748           5
56    5637176749           5
57    5637176750           8
58    5637176751           8
59    5637176752           8
60    5637176753           8
61    5637176754           5
62    5637176755           5
63    5637176756           8
64    5637176757           8
74    5637176785           5
75    5637176786           5
76    5637176787           8
77    5637176788           8
78    5637176789           5
79    5637176790           5
80    5637176791           8
81    5637176792           8
103   5637178626           8
104   5637178627           5
105   5637178629           8
106   5637178636           5
107   5637178637           8
108   5637178638           5
109   5637178650           8
110   5637178651           8
112   5637178655           5
113   5637178656           8
118   5637207387         475
130   5637207382         325
131   5637207383         350
133   5637207385         400
134   5637207386         425
135   5637207388         500
159   5637207425           8
162   5637208178          31
220   5637208946           8
221   5637208947          10
238   5637186197           8
252   5637208199           5
321   5637210867          25
335   5637216766           8
355   5637219889           5
367   5637210975          25
368   5637210976          25
369   5637210977          25
426   5637213447           5
521   5637232541           5
522   5637232542           8
624   5637240854           8
625   5637240855           5
628   5637245187           5
629   5637245188           8
630   5637245189           5
631   5637245190          12
632   5637245191          12
633   5637245193           5
672   5637235790           8
771   5637245502          12
786   5637245334          12
794   5637247545           8
795   5637247546           5
898   5637284605           5
899   5637284607           8
900   5637284608           8
901   5637284609           8
914   5637283741          26
920   5637284448          10
922   5637284458            
930   5637283644          31
941   5637284229           5
953   5637290729           5
957   5637305045           5
958   5637305065          12
959   5637305067           5
976   5637304998          52
977   5637305000          24
978   5637305009          16
979   5637305675           5
980   5637305672           5
981   5637305674           8
982   5637305676           8
1034  5637327748           8
1035  5637327750           5
1036  5637327751           8
1037  5637327754           5
1038  5637327769           5
1054  5637328119           8
1055  5637328126           5
1056  5637328129           8
1061  5637338770           5
1062  5637338771           5
1063  5637338772           8
1064  5637338773           8
1103  5637338793           5
1124  5637352836           8
1129  5637353638           5
1135  5637353561           5
1239  5637380591           8
1244  5637380597           8
1247  5637380604           5
1248  5637380605           5
1260  5637412507           8
1264  5637406680         375
1270  5637417099           5
1273  5637423065           5
1274  5637423066           8
1275  5637423067           5
1276  5637423068           8
1294  5637147194           8
1295  5637147195           8
1296  5637147210          12
1316  5637146959           5
1323  5637146960           8
1340  5637146961           5
1342  5637146962           8
1343  5637146963           5
1344  5637146964           8
138

In [29]:
dia=r'''(?i)(\d+.?tab)|()'''
tab['eigth']=tab['long_desc'].apply(lambda x: re_extract(dia, str(x)))

tab['eigth']=tab['eigth'].apply(lambda x: re.sub(r"(?i)[\-\s•]?tab|7530009594441|\[\'|\'\]|",'',str(x)))
dia=r'''(?i)(\,|\[)|()'''
tab['nine']=tab['eigth'].apply(lambda x: re_extract(dia, str(x)))
na=tab[tab['nine'].astype(str)!='[]']
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
matchna
# tab['nine'].explode().value_counts()

external_id Q:tab_count
0     5637178488         n/a
1     5637178506         n/a
2     5637178507         n/a
3     5637178508         n/a
4     5637178515         n/a
...          ...         ...
2264  5637584952         n/a
2265  5637584953         n/a
2266  5637584955         n/a
2267  5637584956         n/a
2268  5637585034         n/a

[2005 rows x 2 columns]

In [33]:
# # def get_df_name(df):
# #     name =[x for x in globals() if globals()[x] is df][0]
# #     return name
# # print(get_df_name(matchsplitinch))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchcustom))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchesna))


# # def get_df_name(df):
# #     name =[x for x in globals() if globals()[x] is df][0]
# #     return name
# # print(get_df_name(matchesheight))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matchnote))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matchtab))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matchna))

matchna


In [ ]:
stop 

# send to the folder for upload

In [23]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchtab)

In [35]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchna)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matcheight)

In [24]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchone)

In [65]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchnote)

In [27]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchpaper)